In [0]:
from azure.mgmt.resource import SubscriptionClient
from azure.identity import *
credential = DefaultAzureCredential()
subscription_client = SubscriptionClient(credential)
subs = list(subscription_client.subscriptions.list())

if subs:
    subscriptionId = subs[0].subscription_id
else:
    print("No subscriptions found. Does the security principal have the Reader role for at least 1 subscription?")

In [0]:
import requests
url = f"https://management.azure.com/subscriptions/{subscriptionId}/locations?api-version=2022-12-01"

# Set the authorization header
headers = {
    "Authorization": f"Bearer {credential.get_token('https://management.azure.com/.default').token}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers = headers)
data = response.json()


In [0]:
df = spark.read.json(spark.sparkContext.parallelize([data]))
df_flattened = df.selectExpr("explode(value) as location").select("location.*").where("metadata.regionType == 'Physical'").selectExpr("name", "metadata.regionCategory as regionCategory", "metadata.physicalLocation as dataCenterLocation", "size(availabilityZoneMappings) as numZones","metadata.longitude as longitude", "metadata.latitude as latitude", "metadata.pairedRegion.name[0] as pairedRegion")

display(df_flattened)

name,regionCategory,dataCenterLocation,numZones,longitude,latitude,pairedRegion
eastus,Recommended,Virginia,3,-79.8164,37.3719,westus
southcentralus,Recommended,Texas,3,-98.5,29.4167,northcentralus
westus2,Recommended,Washington,3,-119.852,47.233,westcentralus
westus3,Recommended,Phoenix,3,-112.074036,33.448376,eastus
australiaeast,Recommended,New South Wales,3,151.2094,-33.86,australiasoutheast
southeastasia,Recommended,Singapore,3,103.833,1.283,eastasia
northeurope,Recommended,Ireland,3,-6.2597,53.3478,westeurope
swedencentral,Recommended,Gävle,3,17.14127,60.67488,swedensouth
uksouth,Recommended,London,3,-0.799,50.941,ukwest
westeurope,Recommended,Netherlands,3,4.9,52.3667,northeurope


In [0]:
from pyspark.sql.functions import *
# Create a lookup DataFrame for paired regions
paired_df = df_flattened.selectExpr("name as pairedRegionName", "longitude as pairedLongitude", "latitude as pairedLatitude"
)

# Perform a self join to add the pairedLongitude and pairedLatitude columns
df_with_paired_location = df_flattened.join(
    paired_df,
    df_flattened.pairedRegion == paired_df.pairedRegionName,
    "left"
).drop("pairedRegionName")

# Display the DataFrame
display(df_with_paired_location)

name,regionCategory,dataCenterLocation,numZones,longitude,latitude,pairedRegion,pairedLongitude,pairedLatitude
eastasia,Recommended,Hong Kong,3,114.188,22.267,southeastasia,103.833,1.283
canadaeast,Other,Quebec,-1,-71.217,46.817,canadacentral,-79.383,43.653
westeurope,Recommended,Netherlands,3,4.9,52.3667,northeurope,-6.2597,53.3478
southafricawest,Other,Cape Town,-1,18.843266,-34.075691,southafricanorth,28.21837,-25.73134
centralindia,Recommended,Pune,3,73.9197,18.5822,southindia,80.1636,12.9822
norwayeast,Recommended,Norway,3,10.752245,59.913868,norwaywest,5.733107,58.969975
westindia,Other,Mumbai,-1,72.868,19.088,southindia,80.1636,12.9822
switzerlandnorth,Recommended,Zurich,3,8.564572,47.451542,switzerlandwest,6.143158,46.204391
uksouth,Recommended,London,3,-0.799,50.941,ukwest,-3.084,53.427
switzerlandwest,Other,Geneva,-1,6.143158,46.204391,switzerlandnorth,8.564572,47.451542


In [0]:
df_with_distance = df_with_paired_location.withColumn("distance", round(6371 * acos(cos(radians(df_with_paired_location.latitude)) * cos(radians(df_with_paired_location.pairedLatitude)) * cos(radians(df_with_paired_location.longitude) - radians(df_with_paired_location.pairedLongitude)) + sin(radians(df_with_paired_location.latitude)) * sin(radians(df_with_paired_location.pairedLatitude))), 2))

display(df_with_distance.orderBy("regionCategory", "distance", ascending=False))

name,regionCategory,dataCenterLocation,numZones,longitude,latitude,pairedRegion,pairedLongitude,pairedLatitude,distance
brazilsouth,Recommended,Sao Paulo State,3,-46.633,-23.55,southcentralus,-98.5,29.4167,8087.47
eastus,Recommended,Virginia,3,-79.8164,37.3719,westus,-122.417,37.783,3721.23
westus3,Recommended,Phoenix,3,-112.074036,33.448376,eastus,-79.8164,37.3719,2941.27
southeastasia,Recommended,Singapore,3,103.833,1.283,eastasia,114.188,22.267,2588.18
eastasia,Recommended,Hong Kong,3,114.188,22.267,southeastasia,103.833,1.283,2588.18
southcentralus,Recommended,Texas,3,-98.5,29.4167,northcentralus,-87.6278,41.8819,1695.75
centralus,Recommended,Iowa,3,-93.6208,41.5908,eastus2,-78.3889,36.6681,1420.77
southafricanorth,Recommended,Johannesburg,3,28.21837,-25.73134,southafricawest,18.843266,-34.075691,1293.91
westus2,Recommended,Washington,3,-119.852,47.233,westcentralus,-110.234,40.89,1041.69
centralindia,Recommended,Pune,3,73.9197,18.5822,southindia,80.1636,12.9822,913.05
